In [28]:
import pandas as pd
import numpy as np
import random
import copy

data = pd.read_csv('/home/roshan/Documents/FreelanceWork/2017-10-17 Bayesian Networks/iris.data',header=None)
data.columns = ['x1','x2','x3','x4','Class']


In [29]:
label = data['Class']
data = data.drop('Class',axis = 1)

In [30]:
data.iloc[131]

x1    7.9
x2    3.8
x3    6.4
x4    2.0
Name: 131, dtype: float64

In [31]:
# Equal Width Bins
for col in data.columns:
    data[col] = pd.cut(data[col],bins=3,labels=['0','1','2']).astype('int')   

# Equal Frequency Bins
#for col in data.columns:
#    limits = list(data[col].quantile([0.0,0.33,0.67,1.0]))
#    limits[3] = limits[3]+0.2
#    print(limits)
#    data[col] = pd.cut(data[col],bins=limits,labels=['0','1','2'],right=False).astype('int')   

    
    
    

In [32]:
label[label=='Iris-setosa']=0
label[label=='Iris-versicolor']=1
label[label=='Iris-virginica']=2

data['Class'] = label.astype(int)
data.head()

,x1,x2,x3,x4,Class
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


In [33]:
x=np.repeat(list(range(10)),repeats=15)
data['fold'] = pd.Series(x)
data.describe()

,x1,x2,x3,x4,Class,fold
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.740000,0.786667,0.973333,0.973333,1.000000,4.500000
std,0.679814,0.608515,0.802234,0.802234,0.819232,2.881904
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,4.500000
75%,1.000000,1.000000,2.000000,2.000000,2.000000,7.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,9.000000


In [34]:
class Node(object):
    '''
    Defines a Node Class for storing characteristics and CPT of each node
    '''
    
    def __init__(self,name):
        self.parents = []
        self.children = []
        self.name = name
        self.cpt=[]
        self.limit = 3
        
    def addParent(self,x):
        self.parents.append(x)
    
    def addChild(self,x):
        self.children.append(x)
    
    def createCPT(self,data):
        cpt = computeProb(data,self.limit,self.parents,self.name)
        self.cpt = cpt

In [35]:
def computeProb(data,limit,cols,target):
    
    numCol = len(cols)
    
    if numCol==0:
        return(cpt_0(data,limit,cols,target))
    elif numCol ==1:
        return(cpt_1(data,limit,cols,target))
    elif numCol ==2:
        return(cpt_2(data,limit,cols,target))
    elif numCol ==3:
        return(cpt_3(data,limit,cols,target))
    else:
        return(cpt_4(data,limit,cols,target))
            

In [82]:
#Functions for computing the Conditional Probability Tables (CPTs)

def cpt_2(data,limit,cols,target):
    
    cpt = []
    alpha = 0.001
    
    for var1 in range(limit):
        for var2 in range(limit):
            
            totalN = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) ] )
            
            for targetVar in range(limit):
                
                count = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) & (data[target]==targetVar) ] )
                if totalN ==0:
                    cpt.append([var1,var2,targetVar, float(totalN + 3*alpha)])
                else:
                    cpt.append([var1,var2,targetVar, float(count)/float(totalN + 3*alpha)])
                    
    cpt = pd.DataFrame(cpt, columns=[cols[0],cols[1],target,'prob'])
                
    return(cpt)

def cpt_1(data,limit,cols,target):
    
    cpt = []
    alpha = 0.001
    
    for var1 in range(limit):
            
        
        totalN = len( data[ (data[cols[0]]==var1)] )
        
            
        for targetVar in range(limit):
            
            count = len( data[ (data[cols[0]]==var1) & (data[target]==targetVar) ] )
            
            if totalN ==0:
                cpt.append([var1,targetVar, float(totalN + 3*alpha)])
            else:
                cpt.append([var1,targetVar, float(count)/float(totalN + 3*alpha)])
                    
    cpt = pd.DataFrame(cpt, columns=[cols[0],target,'prob'])
                
    return(cpt)

def cpt_0(data,limit,cols,target):
    
    alpha = 0.001
    cpt = []
    
    
    totalN = len( data )
    
            
    for targetVar in range(limit):
            
        count = len( data[ (data[target]==targetVar) ] )
        if totalN ==0:
            cpt.append([targetVar, alpha/float(totalN + 3*alpha)])
        else:
            cpt.append([targetVar, float(count)/(totalN + 3*alpha)])
                    
    cpt = pd.DataFrame(cpt, columns=[target,'prob'])
                
    return(cpt)


def cpt_3(data,limit,cols,target):
    
    cpt = []
    alpha = 0.001
    
    for var1 in range(limit):
        for var2 in range(limit):
            for var3 in range(limit):
            
                totalN = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) & (data[cols[2]]==var3) ] )

                for targetVar in range(limit):

                    count = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) & (data[cols[2]]==var3) & (data[target]==targetVar) ] )
                    if totalN ==0:
                        cpt.append([var1,var2,var3,targetVar, alpha/float(totalN + 3*alpha)])
                    else:
                        cpt.append([var1,var2,var3,targetVar, float(count)/float(totalN + 3*alpha)])
                    
    cpt = pd.DataFrame(cpt, columns=[cols[0],cols[1],cols[2],target,'prob'])
                
    return(cpt)

def cpt_4(data,limit,cols,target):
    
    cpt = []
    alpha = 0.001
    
    for var1 in range(limit):
        for var2 in range(limit):
            for var3 in range(limit):
                for var4 in range(limit):
            
                    totalN = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) & (data[cols[2]]==var3) & (data[cols[3]]==var4) ] )

                    for targetVar in range(limit):

                        count = len( data[ (data[cols[0]]==var1) & (data[cols[1]]==var2) & (data[cols[2]]==var3) & (data[cols[3]]==var4) & (data[target]==targetVar) ] )
                        if totalN ==0:
                            cpt.append([var1,var2,var3,var4,targetVar, alpha/float(totalN + 3*alpha)])
                        else:
                            cpt.append([var1,var2,var3,var4,targetVar, float(count)/float(totalN + 3*alpha)])

    cpt = pd.DataFrame(cpt, columns=[cols[0],cols[1],cols[2],cols[3],target,'prob'])
                
    return(cpt)

In [37]:
structMap = {0:[1,2],1:[1,3],2:[1,4],3:[2,3],4:[2,4],5:[3,4]}   # Mapping of the structure position and the nodes that it connects


class BayesNet(object):
    
    def __init__(self,numNodes,structure):
        self.structure = structure  # Array that defines the structure of the Bayes Net
        self.numNodes = numNodes
        self.varNodes={}
        self.classNode=0
        
        
    def initGraph(self):
        '''
        Initializes components of the Bayes Net Graph
        '''
        
        self.classNode = Node('Class')
        
        for i in range(self.numNodes):
            self.varNodes['x'+str(i+1)]=Node('x'+str(i+1))
            self.varNodes['x'+str(i+1)].parents.append('Class')
        
        for i in range(len(self.structure)):
            
            edgeNodes = structMap[i]
            firstNode = 'x'+str(edgeNodes[0])
            secondNode = 'x'+str(edgeNodes[1])
            
            if self.structure[i]==1:
                self.varNodes[firstNode].children.append(secondNode)
                self.varNodes[secondNode].parents.append(firstNode)
            elif self.structure[i]==-1:
                self.varNodes[secondNode].children.append(firstNode)
                self.varNodes[firstNode].parents.append(secondNode)
    
    def compCPT(self,data):
        '''
        Computes Conditional Probability Table for all the nodes
        '''
        
        self.classNode.createCPT(data)
        
        for i in range(len(self.varNodes)):
            self.varNodes['x'+str(i+1)].createCPT(data)
            
    
    def predict(self,data):
        '''
        Predicts most likely class given a single data sample
        
        '''
        maxProb = 0
        maxProbClass = 0

        for classVal in range(3):

            dt = data.copy()
            dt["Class"] = classVal
            prob = 1.0

            for i in range(self.numNodes):
                #print('Node is x'+str(i+1))

                pt=self.varNodes['x'+str(i+1)].cpt

                mergeList = self.varNodes['x'+str(i+1)].parents + ['x'+str(i+1)]

                cpt_prob = pd.merge(left=pt,right=dt,on=mergeList,how='inner')['prob'][0]
                #print("cpt_prob is ",str(cpt_prob))

                prob = cpt_prob*prob

            #print("Class :%d Prob : %f"%(classVal,prob))

            if prob>maxProb:
                maxProb = prob
                maxProbClass = classVal
                
        return(maxProbClass)

            

In [38]:
structure=[-1,-1,0,0,0,0]
net = BayesNet(4,structure)
net.initGraph()


In [39]:
net.compCPT(data)

In [40]:
net.varNodes['x1'].cpt

,Class,x2,x3,x1,prob
0,0,0,0,0,0.997009
1,0,0,0,1,0.000000
2,0,0,0,2,0.000000
3,0,0,1,0,0.333333
4,0,0,1,1,0.333333
5,0,0,1,2,0.333333
6,0,0,2,0,0.333333
7,0,0,2,1,0.333333
8,0,0,2,2,0.333333
9,0,1,0,0,0.999917


In [41]:
errors = 0

#Testing on the overall dataset

for i in range(150):
    
    y = data.iloc[i:(i+1)]
    out = net.predict(y)
    
    if out != label[i]:
        errors+=1
        
print(errors)

4


In [42]:
errors

4

In [43]:
data.head()

,x1,x2,x3,x4,Class,fold
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0


In [254]:
for fold in range(10):
    print("Fold is %d"%fold)
    
    train = data[data['fold']!=fold]
    test = data[data['fold']==fold]
   
    
    train.drop('fold',axis = 1, inplace=True)
    test.drop('fold',axis = 1, inplace=True)
    
    net = BayesNet(4)
    net.initGraph()
    net.compCPT(train)
    errors = 0

    #Testing

    for i in range(len(test)):

        y = test.iloc[i:(i+1)]
   
        #yLabel = test.iloc[i]['Class']
        out = net.predict(y)
 
    

        if out != test.iloc[i]['Class']:
            errors+=1

    print(errors)
    print("---------------------------------")
    

Fold is 0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0
---------------------------------
Fold is 1
0
---------------------------------
Fold is 2
0
---------------------------------
Fold is 3
0
---------------------------------
Fold is 4
1
---------------------------------
Fold is 5
2
---------------------------------
Fold is 6
0
---------------------------------
Fold is 7
1
---------------------------------
Fold is 8
2
---------------------------------
Fold is 9
0
---------------------------------


In [48]:
def kFold(data,k=10,structure=[0,0,0,0,0,0],verbose=True):
    '''
    Carries out 10-fold CV
    '''
    
    x=np.repeat(list(range(k)),repeats=(len(data)/k))
    data['fold'] = pd.Series(x)
    
    foldSize = len(data)/k
    accuracyList = []
    
    for fold in range(k):
        

        train = data[data['fold']!=fold]
        test = data[data['fold']==fold]

        train.drop('fold',axis = 1, inplace=True)
        test.drop('fold',axis = 1, inplace=True)

        net = BayesNet(4,structure)
        net.initGraph()
        net.compCPT(train)
        errors = 0

        #Testing
        for i in range(len(test)):

            y = test.iloc[i:(i+1)]
            out = net.predict(y)

            if out != test.iloc[i]['Class']:
                errors+=1
        acc = float(foldSize - errors)/foldSize
        accuracyList.append(acc)
        
        if verbose==True:
            print("Fold :%d Accuracy : %f"%(fold,acc))
    
    if verbose ==True:
        print("Overall CV accuracy : %f"%(np.mean(accuracyList)))
    
    return(np.mean(accuracyList))


In [255]:
kFold(data,10)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Fold :0 Accuracy : 1.000000
Fold :1 Accuracy : 1.000000
Fold :2 Accuracy : 1.000000
Fold :3 Accuracy : 1.000000
Fold :4 Accuracy : 0.933333
Fold :5 Accuracy : 0.866667
Fold :6 Accuracy : 1.000000
Fold :7 Accuracy : 0.933333
Fold :8 Accuracy : 0.866667
Fold :9 Accuracy : 1.000000
Overall CV accuracy : 0.960000


## Genetic Algorithm 

In [84]:
class Gene(object):
    
    def __init__(self,structure):
        self.structure = structure
        self.fitness = 0.0

def mutate(gene,pm):
    '''
    Randomly mutates segments of the structure based on the probability pm
    '''
    
    structure = copy.deepcopy(gene.structure)
    
    for i in range(len(structure)):
        
        if random.random()>pm:
            
            if structure[i]==0:
                structure[i] =random.choice([-1,1])
            elif structure[i] ==1:
                structure[i] =random.choice([-1,0])
            else:
                structure[i] =random.choice([1,0])
    
    mutatedGene = Gene(structure)
    return(mutatedGene)
                

def crossover(gene1,gene2):
    '''
    Computes a new structure based on the crossover of two parent structures
    '''
    structure1 = gene1.structure
    structure2 = gene2.structure
    
    
    crossoverGene = Gene(structure1[0:3]+structure2[3:6])
    return(crossoverGene)
    
    
    
    
def generatePop(n):
    '''
    Randomly generates population of size n
    '''
    
    
    popList = []
    
    for i in range(n):
        
        structure = []
        for pos in range(6):
            allele = random.choice([-1,0,1])
            structure.append(allele)
        
        popList.append(Gene(structure))
            
    return(popList)


def evaluateGene(data,gene):
    
    acc = kFold(data,structure=gene.structure,verbose=False)
    
    gene.fitness = acc
    return(acc)

def evaluatePop(data,pop):
    
    for i in range(len(pop)):
        acc = kFold(data,structure=pop[i].structure,verbose=False)
        pop[i].fitness = acc
        print("Gene : %d Fitness : %f"%(i,acc))
    
def runGA(data,popSize=5, pm = 0.7, crossoverLimit= 3, nIterations=3):
    
    #Generating a population
    pop = generatePop(popSize)
    
    evaluatePop(data,pop)
    pop = sorted(pop, key = lambda x : x.fitness, reverse=True)
    
    print("Best score from raw batch : %f",%pop[0].fitness)
    
    
    for iteration in range(nIterations):
        print("Iteration %d"%iteration)
        
        mutateList = []
        
        for i in pop:
            mutatedGene = mutate(i,pm)
            mutateList.append(mutatedGene)
            
        
        evaluatePop(data,mutateList)
        for x in mutateList:
            print("structure : %s fitness : %f"%(x.structure,x.fitness))
        
        crossoverList = []
        
        for parent1 in range(crossoverLimit-1):
            for parent2 in range(parent1+1,crossoverLimit):
                
                crossoverGene = crossover(pop[parent1],pop[parent2])
                
                crossoverList.append(crossoverGene)
            
        evaluatePop(data,crossoverList)
        for x in crossoverList:
            print("structure : %s fitness : %f"%(x.structure,x.fitness))
        
        pop = pop+mutateList+crossoverList
        pop = sorted(pop, key = lambda x : x.fitness, reverse=True)
        pop = pop[0:10]
    
    print("The best structure : %s Fitness : %f"%(pop[0].structure,pop[0].fitness))
    
    
    
    
    
    
    

In [85]:
runGA(data)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Gene : 0 Fitness : 0.913333
Gene : 1 Fitness : 0.946667
Gene : 2 Fitness : 0.926667
Gene : 3 Fitness : 0.933333
Gene : 4 Fitness : 0.933333
('Best score from raw batch : %f', 0.94666666666666666)
Iteration 0
Gene : 0 Fitness : 0.946667
Gene : 1 Fitness : 0.920000
Gene : 2 Fitness : 0.913333
Gene : 3 Fitness : 0.920000
Gene : 4 Fitness : 0.913333
structure : [0, -1, -1, -1, -1, -1] fitness : 0.946667
structure : [-1, -1, 0, 1, -1, 0] fitness : 0.920000
structure : [-1, 0, 0, -1, 1, 1] fitness : 0.913333
structure : [1, 1, -1, 0, -1, 1] fitness : 0.920000
structure : [-1, 1, 1, -1, -1, -1] fitness : 0.913333
Gene : 0 Fitness : 0.946667
Gene : 1 Fitness : 0.966667
Gene : 2 Fitness : 0.966667
structure : [0, -1, -1, 1, -1, -1] fitness : 0.946667
structure : [0, -1, -1, -1, 1, 0] fitness : 0.966667
structure : [0, -1, 0, -1, 1, 0] fitness : 0.966667
Iteration 1
Gene : 0 Fitness : 0.960000
Gene : 1 Fitness : 0.966667
Gene : 2 Fitness : 0.920000
Gene : 3 Fitness : 0.946667
Gene : 4 Fitness : 

In [46]:
for i in range(5):
    acc = evaluateGene(data,pop[i])
    print(acc)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Fold :0 Accuracy : 1.000000
Fold :1 Accuracy : 1.000000
Fold :2 Accuracy : 1.000000
Fold :3 Accuracy : 1.000000
Fold :4 Accuracy : 0.933333
Fold :5 Accuracy : 0.866667
Fold :6 Accuracy : 1.000000
Fold :7 Accuracy : 0.866667
Fold :8 Accuracy : 0.600000
Fold :9 Accuracy : 1.000000
Overall CV accuracy : 0.926667
0.926666666667
Fold :0 Accuracy : 1.000000
Fold :1 Accuracy : 1.000000
Fold :2 Accuracy : 1.000000
Fold :3 Accuracy : 0.866667
Fold :4 Accuracy : 0.933333
Fold :5 Accuracy : 0.733333
Fold :6 Accuracy : 1.000000
Fold :7 Accuracy : 0.933333
Fold :8 Accuracy : 0.933333
Fold :9 Accuracy : 0.933333
Overall CV accuracy : 0.933333
0.933333333333
Fold :0 Accuracy : 1.000000
Fold :1 Accuracy : 1.000000
Fold :2 Accuracy : 1.000000
Fold :3 Accuracy : 1.000000
Fold :4 Accuracy : 0.933333
Fold :5 Accuracy : 0.866667
Fold :6 Accuracy : 1.000000
Fold :7 Accuracy : 0.933333
Fold :8 Accuracy : 0.733333
Fold :9 Accuracy : 1.000000
Overall CV accuracy : 0.946667
0.946666666667
Fold :0 Accuracy : 1.0

In [78]:
for i in range(4):
    for j in range(i+1,5):
        print(i,j)

(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [278]:
help(random.random)

Help on built-in function random:

random(...)
    random() -> x in the interval [0, 1).



In [73]:
a = Gene([1,1,1,1,1,1])

In [65]:
st =a.structure

In [66]:
st[1]=0

In [67]:
a.structure

[1, 0, 1, 1, 1, 1]

In [74]:
import copy

st = copy.deepcopy(a.structure)

In [75]:
st[1]=0

In [76]:
a.structure

[1, 1, 1, 1, 1, 1]

In [77]:
st

[1, 0, 1, 1, 1, 1]